# 2 Error mitigation

## 2.1 We used VQE to calculate the energy values of the following three quantum circuits, and linearly fitted the obtained energy values, extrapolating them to the energy values without noise.
<font size=5>

* Scale one: 
<div align=left><img src="IMG/scale_one.png" width=800 length=800></div>

* Scale three: 
<div align=left><img src="IMG/scale_three.png" width=800 length=800></div>

* Scale five: 
<div align=left><img src="IMG/scale_five.png" width=800 length=800></div>

In [1]:
from datetime import datetime

import numpy as np
import time

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper,ParityMapper,QubitConverter
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit.algorithms.optimizers import *  


from Estimator_with_noise import estimator_noise
from Ansatz import Adapt_Givens_Ansatz
from Noise_models import fakecairo, fakekolkata, fakemontreal

/home/dev/anaconda3/envs/Qikist_Chemistry/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [ ]:
# Set up molecule
ultra_simplified_ala_string = """
O 0.0 0.0 0.0
H 0.45 -0.1525 -0.8454
"""
driver = PySCFDriver(
    atom=ultra_simplified_ala_string.strip(),
    basis='sto3g',
    charge=1,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)
qmolecule = driver.run()
mapper = JordanWignerMapper()

# Set up VQE
seed_pool = [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]
# seed_pool = [20]
noise_word = ['fakecairo','fakekolkata','fakemontreal']
# ansatz = Adapt_Givens_Ansatz()

Scale_one = Adapt_Givens_Ansatz()
Scale_one_inverse = Scale_one.inverse()

indentity_one_layer = Scale_one.compose(Scale_one_inverse)

indentity_two_layer = indentity_one_layer.compose(indentity_one_layer)

Scale_three = indentity_one_layer.compose(Scale_one)

Scale_five =  indentity_two_layer.compose(Scale_one)


optimizer = COBYLA(maxiter=100)
ground_energy = -74.38714627
shots = 6000


In [2]:
# Computed the ground energy with Scale_one

ansatz = Scale_one

for index, noise in enumerate(noise_word):
    noise_model = globals()[noise]()
    loss = []
    for seed in seed_pool:
        
        print(f'\n Seed = {seed}, Noise model = {noise}')
        
        # Callback to store intermediate results during optimization
        parameters_list = []
        values = []
        last_call_time = None

        def callback(eval_count, parameters, mean, std):
            global last_call_time
            current_call_time = datetime.now()
            if last_call_time is not None:
                print(f"Cost Time: {current_call_time - last_call_time}")

            last_call_time = current_call_time
            parameters_list.append(parameters)
            values.append(mean)
            print(f'iter: {len(parameters_list)}, loss: {mean}, params: {parameters}')
        
        estimator = estimator_noise(seed, shots, noise_model)
        
        vqe_solver = VQE(estimator, ansatz, optimizer, callback=callback)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        start_time = time.time()
        res = calc.solve(qmolecule)
        end_time = time.time()
        
        cost_time = end_time - start_time
        result = res.computed_energies + res.nuclear_repulsion_energy
        error_rate = abs((ground_energy - result) / ground_energy * 100)
        
        with open(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_one.txt'.format(),"a") as f:
            f.write(f'Noise_model: {noise}, seeds = {seed},Time = {cost_time}, '
                     f'energy = {res.computed_energies}, Error rate: {error_rate},parameters = {parameters_list[-1]} '
                     f'optimizer = COBYLA, Ansatz = Adapt_Givens, values = {values} \n')
    np.save(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_one.npy',loss)   



 Seed = 20, Noise model = fakecairo


/home/dev/anaconda3/envs/Qikist_Chemistry/lib/python3.11/site-packages/qiskit_nature/deprecation.py:297: PauliSumOpDeprecationWarning: PauliSumOp is deprecated as of version 0.6.0 and support for them will be removed no sooner than 3 months after the release. Instead, use SparsePauliOp. You can switch to SparsePauliOp immediately, by setting `qiskit_nature.settings.use_pauli_sum_op` to `False`.
  return func(*args, **kwargs)


iter: 1, loss: -78.09897168525542, params: [0.0, 0.0]
Cost Time: 0:01:03.721618
iter: 2, loss: -77.82979770285353, params: [1.0, 0.0]
Cost Time: 0:01:03.654040
iter: 3, loss: -73.86071177102907, params: [0.0, 1.0]
Cost Time: 0:01:04.225094
iter: 4, loss: -73.87198389930008, params: [-0.063382793134161, -0.9979892892884734]
Cost Time: 0:01:03.547158
iter: 5, loss: -76.87966211753897, params: [-0.0316913965670805, -0.4989946446442367]
Cost Time: 0:01:04.163631
iter: 6, loss: -77.84216968097931, params: [-0.027185921689837227, 0.2485174554470451]
Cost Time: 0:01:04.474801
iter: 7, loss: -78.09854594995664, params: [-0.12425872772352255, -0.013592960844918614]
Cost Time: 0:01:03.922007
iter: 8, loss: -77.84155361384266, params: [0.02800976949614177, -0.24842595036101403]
Cost Time: 0:01:04.421536
iter: 9, loss: -78.03877444549668, params: [0.014004884748070885, -0.12421297518050702]
Cost Time: 0:01:03.710987
iter: 10, loss: -78.07945628290534, params: [-0.006356934235980162, 0.062175874638

In [3]:
# Computed the ground energy with Scale_three
ansatz = Scale_three

for index, noise in enumerate(noise_word):
    noise_model = globals()[noise]()
    
    loss = []
    
    for seed in seed_pool:
        print(f'\n Seed = {seed}, Noise model = {noise}')
        
        # Callback to store intermediate results during optimization
        parameters_list = []
        values = []
        last_call_time = None


        def callback(eval_count, parameters, mean, std):
            global last_call_time
            current_call_time = datetime.now()
            if last_call_time is not None:
                print(f"Cost Time: {current_call_time - last_call_time}")

            last_call_time = current_call_time
            parameters_list.append(parameters)
            values.append(mean)
            print(f'iter: {len(parameters_list)}, loss: {mean}, params: {parameters}')
        
        estimator = estimator_noise(seed, shots, noise_model)
        
        vqe_solver = VQE(estimator, ansatz, optimizer, callback=callback)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        start_time = time.time()
        res = calc.solve(qmolecule)
        end_time = time.time()
        
        cost_time = end_time - start_time
        result = res.computed_energies + res.nuclear_repulsion_energy
        error_rate = abs((ground_energy - result) / ground_energy * 100)
        
        loss.append(res.computed_energies)
        with open(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_three.txt'.format(),"a") as f:
            f.write(f'Noise_model: {noise}, seeds = {seed},Time = {cost_time}, '
                     f'energy = {res.computed_energies}, Error rate: {error_rate},parameters = {parameters_list[-1]} '
                     f'optimizer = COBYLA, Ansatz = Adapt_Givens, values = {values}  \n')
        
    np.save(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_three.npy',loss)     



 Seed = 20, Noise model = fakecairo
iter: 1, loss: -78.05994987699741, params: [0.0, 0.0]
Cost Time: 0:01:56.232455
iter: 2, loss: -77.83440770120853, params: [1.0, 0.0]
Cost Time: 0:01:56.393229
iter: 3, loss: -73.80380446595932, params: [0.0, 1.0]
Cost Time: 0:01:58.098423
iter: 4, loss: -73.81307491570264, params: [-0.05291787277670173, -0.9985988677846519]
Cost Time: 0:01:55.913161
iter: 5, loss: -76.8843127716093, params: [-0.026458936388350864, -0.49929943389232595]
Cost Time: 0:01:56.431875
iter: 6, loss: -77.78116090562247, params: [-0.0237188250360475, 0.2488722912236502]
Cost Time: 0:01:56.363099
iter: 7, loss: -78.05650084380768, params: [-0.12443614561182513, -0.011859412518023752]
Cost Time: 0:01:56.731686
iter: 8, loss: -77.78271094431847, params: [0.02986752271398539, -0.24820945003550846]
Cost Time: 0:01:56.425400
iter: 9, loss: -77.98434030707232, params: [0.014933761356992694, -0.12410472501775423]
Cost Time: 0:01:56.932174
iter: 10, loss: -78.03040578251306, params:

In [4]:
# Computed the ground energy with Scale_five

ansatz = Scale_five

for index, noise in enumerate(noise_word):
    noise_model = globals()[noise]()
    loss = []
    for seed in seed_pool:
        print(f'\n Seed = {seed}, Noise model = {noise}')
        
        # Callback to store intermediate results during optimization
        parameters_list = []
        values = []
        last_call_time = None


        def callback(eval_count, parameters, mean, std):
            global last_call_time
            current_call_time = datetime.now()
            if last_call_time is not None:
                print(f"Cost Time: {current_call_time - last_call_time}")

            last_call_time = current_call_time
            parameters_list.append(parameters)
            values.append(mean)
            print(f'iter: {len(parameters_list)}, loss: {mean}, params: {parameters}')
        
        estimator = estimator_noise(seed, shots, noise_model)
        
        vqe_solver = VQE(estimator, ansatz, optimizer, callback=callback)
        vqe_solver.initial_point = [0.0] * ansatz.num_parameters
        
        calc = GroundStateEigensolver(mapper, vqe_solver)
        
        start_time = time.time()
        res = calc.solve(qmolecule)
        end_time = time.time()
        
        cost_time = end_time - start_time
        result = res.computed_energies + res.nuclear_repulsion_energy
        error_rate = abs((ground_energy - result) / ground_energy * 100)
        
        with open(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_five.txt'.format(),"a") as f:
            f.write(f'Noise_model: {noise}, seeds = {seed},Time = {cost_time}, '
                     f'energy = {res.computed_energies}, Error rate: {error_rate},parameters = {parameters_list[-1]} '
                     f'optimizer = COBYLA, Ansatz = Adapt_Givens, values = {values} \n')
    np.save(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_five.npy',loss)   


 Seed = 20, Noise model = fakecairo
iter: 1, loss: -77.88380910646032, params: [0.0, 0.0]
Cost Time: 0:03:00.449806
iter: 2, loss: -77.61115027133852, params: [1.0, 0.0]
Cost Time: 0:02:58.598590
iter: 3, loss: -73.91536282161374, params: [0.0, 1.0]
Cost Time: 0:02:58.134105
iter: 4, loss: -73.93583595283863, params: [-0.0685450998012895, -0.9976480187386889]
Cost Time: 0:02:58.922521
iter: 5, loss: -76.77499503185254, params: [-0.03427254990064475, -0.49882400936934446]
Cost Time: 0:02:57.680279
iter: 6, loss: -77.61832147070261, params: [-0.030186600324819214, 0.24817084671820264]
Cost Time: 0:02:57.740405
iter: 7, loss: -77.8828467310003, params: [-0.12408542335910132, -0.015093300162409605]
Cost Time: 0:02:56.721126
iter: 8, loss: -77.61607609551585, params: [0.03198496620689866, -0.24794548178328144]
Cost Time: 0:02:57.888721
iter: 9, loss: -77.78562378656109, params: [0.01599248310344933, -0.12397274089164072]
Cost Time: 0:02:55.117319
iter: 10, loss: -77.8547326835181, params: 

#### Data Processing

In [ ]:
for noise in noise_word:
    Scale_one = np.load(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_one.npy')
    Scale_three = np.load(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_three.npy')
    Scale_five = np.load(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_Scale_five.npy')
    zipped = zip(Scale_one, Scale_three, Scale_five)
    new_lists = [list(item) for item in zipped]
    for i in range(new_lists)
    np.save(f'Documents/Error_mitigation_data/Whole_data/{noise}_Adapt_Givens_COBYLA_{seed_pool[i]}.npy',new_lists[i])  